### Neuroevolution:
- Bild -> preprocessing -> CNN -> MLP -> Eingabe (NES controller)
- CNN und MLP basieren auf pytorch
- CNN und MLP werden gemeinsam mutiert, die mutation vom agenten der am weitesten kam wird übernommen
- Distanz vom Start wird (fürs erste) der einzige reward sein

#### ToDo:
- Dynamische Mutation (z.B: erhöhe Mutationsstärke bei Stillstand, Mutationsstärke und strength werden als intervall übergeben aus der zufällig ein wert genommen wird)
- Models müssen gespeichert und geladen werden können
- Gespeicherte models nochmal finetunen, testen, überüfen ob die wirklich konstant gute Ergebnisse haben
- Pupulationsize und ggf. andere Hyperparameter direkt im plot angeben

#### Elitismus:
- die n besten agenten bleiben erhalten
- wähle gesamt-n agenten mit wheel selection aus (also proportional zur fitness haben fittere agenten eine höhrere w.keit ausgewählt zu werden, vermeide das "super Agenten" die w.keitsverteilung zerstören, das rad wird also gewollt oft gedreht und vom sieger wird eine mutation erstellt)
- mutiere die gesamt minus n zufällig ausgewählten agenten die dann teil der nächsten generation sind

#### Nice to have:
- Verbesserung der GPU Auslastung (Hat sich als schwierig herausgestellt, im Moment ist cpu-only schneller!)
- Frame skipping um performance zu erhöhen (Auch schwierig hauptlast ist nicht die entscheidung des agenten sondern die laufende mario umgebung)

In [ ]:
import torch
from src.non_reinforcement.neuroevolution.neuroevolution_trainer import NeuroevolutionTrainer
from src.non_reinforcement.neuroevolution.neuroevolution_net import NeuroevolutionNet
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT, SIMPLE_MOVEMENT

DEVICE = "cpu"  #torch.device("cuda" if torch.cuda.is_available() else "cpu")

cnn_config = [
    {"out_channels": 16, "kernel_size": 8, "stride": 4},
    {"out_channels": 32, "kernel_size": 4, "stride": 2},
]

mlp_config = [
    32 * 9 * 9,  # Must match flattened output of CNN
    256,
    128,
    len(SIMPLE_MOVEMENT)
]

model = NeuroevolutionNet(
    input_channels=1,
    num_actions=len(SIMPLE_MOVEMENT),
    cnn_config=cnn_config,
    mlp_config=mlp_config
)

trainer = NeuroevolutionTrainer(
    base_model=model,
    env_name='SuperMarioBros-v0',
    action_set=SIMPLE_MOVEMENT,
    device=DEVICE,
    video_dir="../media/videos",
    generations=40,
    population_size=8,
    max_steps_per_episode=1000,
    mutation_rate=0.05,
    mutation_strength=0.1,
    wheel_selection_temperature=0.5,
    elitism=2
)

trainer.run()
trainer.record_best_agent()